In [ ]:
# -------------------------------------------------------
# STOCK BUY ADVISOR (Only Future Dates)
# -------------------------------------------------------
# This script uses Facebook Prophet to forecast synthetic
# stock prices and recommends the best future day to buy.
#
# Safe for GitHub – uses no real stock API or real data.
# -------------------------------------------------------

import pandas as pd
import numpy as np
from prophet import Prophet
import datetime

# -------------------------------------------------------
# STEP 1 — CREATE SYNTHETIC STOCK DATA
# -------------------------------------------------------
# You can replace this with real data later if needed.
# -------------------------------------------------------

dates = pd.date_range(start="2026-01-01", end="2027-12-31")

np.random.seed(42)
price = (
    150 +
    (np.arange(len(dates)) * 0.02) +
    10 * np.sin(2 * np.pi * dates.dayofweek / 7) +
    6 * np.sin(2 * np.pi * dates.dayofyear / 365) +
    np.random.normal(0, 3, len(dates))
)

df = pd.DataFrame({"ds": dates, "y": price})

# -------------------------------------------------------
# STEP 2 — TRAIN PROPHET MODEL
# -------------------------------------------------------

model = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True
)
model.fit(df)

# -------------------------------------------------------
# STEP 3 — FORECAST NEXT 365 DAYS
# -------------------------------------------------------

future = model.make_future_dataframe(periods=365)
forecast = model.predict(future)

# -------------------------------------------------------
# STEP 4 — USER INPUT
# -------------------------------------------------------

try:
    budget = float(input("Enter your total budget: "))
    qty = float(input("How many stocks do you want to buy? "))
except ValueError:
    print("Invalid input. Please enter numbers only.")
    exit()

# -------------------------------------------------------
# STEP 5 — FILTER ONLY FUTURE FORECAST DATES
# -------------------------------------------------------

today = datetime.datetime.now().date()
future_forecast = forecast[forecast["ds"] > str(today)].copy()

if future_forecast.empty:
    print("Error: No future forecast available.")
    exit()

# -------------------------------------------------------
# STEP 6 — FIND BEST FUTURE BUY DATE
# -------------------------------------------------------

best_idx = future_forecast["yhat"].idxmin()
best_row = future_forecast.loc[best_idx]

best_date = best_row["ds"].date()
best_price = float(best_row["yhat"])

# -------------------------------------------------------
# STEP 7 — CALCULATIONS
# -------------------------------------------------------

budget_enough = budget >= (best_price * qty)
max_can_buy = int(budget // best_price)

# -------------------------------------------------------
# STEP 8 — OUTPUT RESULT
# -------------------------------------------------------

print("\n===== STOCK BUY RECOMMENDATION =====")
print(f"Best Day to Buy: {best_date}")
print(f"Predicted Price on Best Day: {best_price:.2f}")
print("------------------------------------")
print(f"Your Budget: {budget}")
print(f"Desired Stocks: {qty}")
print(f"Budget Enough?: {'YES' if budget_enough else 'NO'}")
print(f"Max Stocks You Can Buy: {max_can_buy}")
print("====================================")
